# Particle Filters

In [1]:
class Map:
    def __init__(self, world_size, landmarks):
        self.world_size = world_size
        self.landmarks = landmarks

In [7]:
from math import *
import random

class Robot:
    def __init__(self, world_size):
        self.x = random.random() * world_size
        self.y = random.random() * world_size
        self.orientation = random.random() * 2.0 * pi
        self.forward_noise = 0.0
        self.turn_noise = 0.0
        self.sense_noise = 0.0
        
    def set(self, new_x, new_y, new_orientation, world_size):
        if new_x < 0 or new_x >= world_size:
            raise (ValueError, 'X coordinate out of bound')
        if new_y < 0 or new_y >= world_size:
            raise (ValueError, 'Y coordinate out of bound')
        if new_orientation < 0 or new_orientation >= 2 * pi:
            raise (ValueError, 'Orientation must be in [0..2pi]')
        
        self.x = float(new_x)
        self.y = float(new_y)
        self.orientation = float(new_orientation)
        
    def set_noise(self, new_f_noise, new_t_noise, new_s_noise):
        self.forward_noise = float(new_f_noise)
        self.turn_noise = float(new_t_noise)
        self.sense_noise = float(new_s_noise)
        
    def sense(self, landmarks):
        Z = []
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            dist += random.gauss(0.0, self.sense_noise)
            Z.append(dist)
        return Z
    
    def move(self, turn, forward, world_size):
        if forward < 0:
            raise (ValueError, 'Robot cant move backwards')
            
        orientation = self.orientation + float(turn) + random.gauss(0.0, self.turn_noise)
        orientation %= 2 * pi
        
        dist = float(forward) + random.gauss(0.0, self.forward_noise)
        x = self.x + (cos(orientation) * dist)
        y = self.y + (sin(orientation) * dist)
        x %= world_size
        y %= world_size
        
        robot = Robot(world_size)
        robot.set(x, y, orientation, world_size)
        robot.set_noise(self.forward_noise, self.turn_noise, self.sense_noise)
        return robot
    
    def Gaussian(self, mu, sigma, x):
        return exp(- ((mu - x) ** 2) / (sigma ** 2) / 2.0) / sqrt(2.0 * pi * (sigma ** 2))    
    
    def measurement_prob(self, measurement, landmarks):
        prob = 1.0;
        for i in range(len(landmarks)):
            dist = sqrt((self.x - landmarks[i][0]) ** 2 + (self.y - landmarks[i][1]) ** 2)
            prob *= self.Gaussian(dist, self.sense_noise, measurement[i])
        return prob
    
    def __repr__(self):
        return '[x=%.6s y=%.6s orient=%.6s]' % (str(self.x), str(self.y), str(self.orientation))

In [21]:
import random

class ParticleFilter:
    def __init__(self, gmap, num_particle):
        self.gmap = gmap
        self.N = num_particle
        
    def initialize_particles(self, new_f_noise, new_t_noise, new_s_noise):
        particles = []
        for i in range(self.N):
            p = Robot(self.gmap.world_size)
            p.set_noise(new_f_noise, new_t_noise, new_s_noise)
            particles.append(p)
        return particles
        
    def move_praticles(self, particles, turn, forward):
        moved_p = []
        for p in particles:
            particle = p.move(turn, forward, self.gmap.world_size)
            moved_p.append(particle)
        return moved_p
        
    def get_importance_weights(self, measurement, particles):
        weights = []
        for p in particles:
            weights.append(p.measurement_prob(measurement, gmap.landmarks))
        return weights
    
    def resampe_particles(self, particles, weights):
        resampled_p = []
        idx = int(random.random() * self.N)
        beta = 0.0
        w_max = max(weights)
        
        for i in range(self.N):
            beta += random.random() * 2.0 * w_max
            while weights[idx] < beta:
                beta -= weights[idx]
                idx = (idx + 1) % self.N
            resampled_p.append(particles[idx])
        return resampled_p

In [22]:
def eval(r, p):
    sum = 0.0
    for i in range(len(p)):
        dx = (p[i].x - r.x + world_size/2.0) % world_size - (world_size / 2.0)
        dy = (p[i].y - r.y + world_size/2.0) % world_size - (world_size / 2.0)
        err = sqrt(dx * dx + dy * dy)
        sum += err
    return sum / float(len(p))

In [23]:
landmarks  = [[20.0, 20.0], [80.0, 80.0], [20.0, 80.0], [80.0, 20.0]]
world_size = 100.0
gmap = Map(world_size, landmarks)

In [27]:
robot = Robot(gmap.world_size)
robot.move(0.1, 5.0, gmap.world_size)
Z = robot.sense(gmap.landmarks)
#print("Actual distance(between Robot and Landmarks):\n", Z, "\n")
#print("Robot position:\n", robot, "\n")

pf = ParticleFilter(gmap, 1000)
particles = pf.initialize_particles(0.05, 0.05, 5.0)

T = 10
for t in range(T):
    robot = robot.move(0.1, 5.0, gmap.world_size)
    Z = robot.sense(gmap.landmarks)
    print("Robot position:\n", robot, "\n")
    particles = pf.move_praticles(particles, 0.1, 5.0)
    weights = pf.get_importance_weights(Z, particles)
    particles = pf.resampe_particles(particles, weights)
    for i in range(100):
        print(particles[i], " - ", weights[i])
    print("+++++++++++++++++++")

Robot position:
 [x=30.120 y=66.824 orient=4.2805] 

[x=26.568 y=69.377 orient=0.8709]  -  3.380991132324771e-23
[x=35.403 y=69.185 orient=1.0082]  -  7.235021090964097e-16
[x=31.402 y=65.055 orient=0.1394]  -  3.397070442885674e-08
[x=27.862 y=71.398 orient=3.3368]  -  3.169660225456518e-32
[x=37.962 y=71.626 orient=1.9688]  -  1.3772451328001165e-27
[x=26.568 y=69.377 orient=0.8709]  -  2.8144896773935926e-64
[x=19.125 y=67.395 orient=2.2337]  -  9.006171476983835e-46
[x=31.402 y=65.055 orient=0.1394]  -  1.5319364853867073e-18
[x=36.651 y=61.544 orient=4.9044]  -  2.507105136134702e-62
[x=38.128 y=69.095 orient=0.8616]  -  8.790863893496835e-36
[x=23.083 y=67.332 orient=4.1891]  -  6.1961424048706334e-96
[x=28.214 y=71.017 orient=5.2156]  -  1.4320479232359367e-07
[x=28.214 y=71.017 orient=5.2156]  -  1.0776260275427279e-36
[x=27.629 y=73.980 orient=3.2784]  -  2.3392161794588702e-102
[x=24.316 y=63.362 orient=2.2465]  -  3.336878762952809e-57
[x=35.403 y=69.185 orient=1.0082]  -  1

[x=18.621 y=48.023 orient=4.6963]  -  3.289554686743217e-08
[x=20.041 y=47.752 orient=4.7293]  -  3.767938013968994e-06
[x=24.070 y=48.630 orient=5.0632]  -  2.7247506882875688e-05
[x=36.522 y=51.319 orient=5.0049]  -  2.634726093434721e-05
[x=22.535 y=48.011 orient=4.9575]  -  2.545554232979284e-05
[x=22.226 y=48.022 orient=4.8976]  -  2.939029731136324e-07
+++++++++++++++++++
Robot position:
 [x=26.866 y=42.289 orient=4.7805] 

[x=25.395 y=43.997 orient=5.1522]  -  2.5477550990682916e-05
[x=25.395 y=43.997 orient=5.1522]  -  2.754243719715701e-05
[x=25.514 y=44.077 orient=5.1805]  -  2.6077437182891716e-06
[x=25.514 y=44.077 orient=5.1805]  -  2.1531799466266674e-05
[x=25.514 y=44.077 orient=5.1805]  -  1.3400374663168671e-05
[x=24.755 y=43.624 orient=5.0702]  -  1.0760108059998773e-05
[x=20.716 y=42.761 orient=4.8111]  -  3.454474632696725e-05
[x=25.171 y=43.800 orient=5.1167]  -  1.3744384522169244e-06
[x=25.171 y=43.800 orient=5.1167]  -  1.62178965982639e-06
[x=27.504 y=44.829 or

[x=34.615 y=26.688 orient=5.3629]  -  2.9606003476163985e-05
[x=34.615 y=26.688 orient=5.3629]  -  6.354177008089803e-06
[x=32.623 y=27.227 orient=5.6199]  -  2.996256979077243e-05
[x=33.829 y=26.011 orient=5.3121]  -  3.190600050039758e-05
[x=34.153 y=26.205 orient=5.3608]  -  2.4138252946917106e-05
[x=34.877 y=26.596 orient=5.3794]  -  3.430243352724066e-05
[x=33.757 y=25.946 orient=5.3440]  -  3.527943895425595e-05
[x=33.388 y=25.940 orient=5.3455]  -  3.1381850907858094e-05
[x=32.514 y=25.396 orient=5.2204]  -  3.347955957862372e-05
[x=34.988 y=26.411 orient=5.3314]  -  8.263894017104064e-06
[x=33.151 y=25.778 orient=5.4231]  -  3.3213394583612057e-05
[x=33.151 y=25.778 orient=5.4231]  -  3.2719282610537304e-05
+++++++++++++++++++
